# 제로베이스 데이터 취업 스쿨 SQL 과제 1
- 스타벅스 이디야 데이터 분석

### 8문제 총 100점

- 1번 5점
- 2번 5점
- 3번 10점
- 4번 5점
- 5번 15점
- 6번 15점
- 7번 40점
- 8번 5점

### 1 ~ 8번 모두 본 노트북 파일에 답안 작성해서 제출해주세요 :)

---

문제 1.

AWS RDS (MySQL) 에 프로젝트 관련 Database 를 생성하고, 접근 가능한 사용자 계정을 생성하세요.

- Database Name : oneday
- User Name / Password : oneday / 1234

In [33]:
import mysql.connector
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from tqdm.notebook import tqdm
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re

In [2]:
conn = mysql.connector.connect(
    host = 'endpoint',
    port = 3306,
    user = 'admin',
    password = '패스워드'
)

mycursor = conn.cursor()

In [6]:
db_sql = 'create database oneday'
user_sql = 'create user "oneday"@"%" identified by "1234"'
grant_sql = 'grant all on oneday.* to "oneday"@"%"'

In [41]:
mycursor.execute(db_sql)

In [43]:
mycursor.execute(user_sql)

In [44]:
mycursor.execute(grant_sql)

제출 1.
- Database 생성문 조회 결과 : SHOW CREATE DATABASE oneday;
- 사용자 권한 확인 결과 : SHOW GRANT FOR ‘oneday’@‘%’

In [45]:
answer1_sql = 'show create database oneday'
answer2_sql = 'show grants for "oneday"@"%"'

In [46]:
mycursor.execute(answer1_sql)
answer1 = mycursor.fetchall()
for row in answer1:
    print(row)

('oneday', "CREATE DATABASE `oneday` /*!40100 DEFAULT CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci */ /*!80016 DEFAULT ENCRYPTION='N' */")


In [47]:
mycursor.execute(answer2_sql)
answer2 = mycursor.fetchall()
for row in answer2:
    print(row)

('GRANT USAGE ON *.* TO `oneday`@`%`',)
('GRANT ALL PRIVILEGES ON `oneday`.* TO `oneday`@`%`',)


문제 2.

스타벅스 이디야 데이터를 저장할 테이블을 다음의 구조로 생성하세요. (PDF 파일 참고)

In [48]:
brand_sql = ("""
create table coffee_brand
             (
             id int auto_increment,
             name varchar(32),
             primary key (id)
             )
             """)

store_sql = ("""
create table coffee_store
             (
             id int auto_increment,
             brand int,
             name varchar(32),
             gu_name varchar(5),
             address varchar(128),
             lat float,
             lng float,
             primary key (id),
             constraint FK_coffee_store foreign key (brand) references coffee_brand(id)
             )
             """)

In [49]:
mycursor.execute('use oneday;')
mycursor.execute(brand_sql)
mycursor.execute(store_sql)
mycursor.execute('show tables')
tables = mycursor.fetchall()
for row in tables:
    print(row)

('coffee_brand',)
('coffee_store',)


제출 2.
- Table 생성 결과 : Desc COFFEE_BRAND; Desc COFFEE_STORE;

In [50]:
mycursor.execute('desc coffee_brand')
desc1 = mycursor.fetchall()
for row in desc1:
    print(row)

('id', 'int', 'NO', 'PRI', None, 'auto_increment')
('name', 'varchar(32)', 'YES', '', None, '')


In [51]:
mycursor.execute('desc coffee_store')
desc1 = mycursor.fetchall()
for row in desc1:
    print(row)

('id', 'int', 'NO', 'PRI', None, 'auto_increment')
('brand', 'int', 'YES', 'MUL', None, '')
('name', 'varchar(32)', 'YES', '', None, '')
('gu_name', 'varchar(5)', 'YES', '', None, '')
('address', 'varchar(128)', 'YES', '', None, '')
('lat', 'float', 'YES', '', None, '')
('lng', 'float', 'YES', '', None, '')


문제 3.

Python 코드로 COFFEE_BRAND 데이터를 다음과 같이 입력하고 확인하세요. (PDF 파일 참고)

In [52]:
insert_brand_sql = 'insert into coffee_brand values (%s, %s)'
brand_data = [(1, "STARBUCKS"), (2, "EDIYA")]
for data in brand_data:
    mycursor.execute(insert_brand_sql, data)
    print(data)
    conn.commit()

(1, 'STARBUCKS')
(2, 'EDIYA')


제출 3.
- COFFEE_BRAND 조회 결과 : SELECT * FROM COFFEE_BRAND;

In [53]:
mycursor.execute('select * from coffee_brand')
select_result = mycursor.fetchall()
for row in select_result:
    print(row)

(1, 'STARBUCKS')
(2, 'EDIYA')


문제 4.

스타벅스 페이지에 접근하는 코드에서 팝업창이 없는 경우, 팝업창을 닫는 코드에서 에러가 발생합니다. 예외처리 해서 에러
메시지를 출력하고 실행이 중단되지 않도록 수정해주세요.

In [111]:
starbucks_url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(starbucks_url)

In [112]:
driver.maximize_window()

제출 4.
- 팝업 예외처리 코드 & 실행 결과 (ipynb)

매장찾기 화면으로 바로 접근하는 방식이어서 팝업창이 뜨지 않습니다만, 예외처리 형태만 구현해 보겠습니다. 

In [113]:
try:
    driver.find_element(By.CSS_SELECTOR, '.holiday_notice_close a').click()
except Exception as e:
    print(e)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".holiday_notice_close a"}
  (Session info: chrome=119.0.6045.160)



In [56]:
# 시/도 선택화면에서 '서울' 선택
seoul_tag = driver.find_element(By.CSS_SELECTOR, '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a')
seoul_tag.click()

In [57]:
# 구 선택화면에서 '전체' 선택
all_tag = driver.find_element(By.CSS_SELECTOR, '#mCSB_2_container > ul > li:nth-child(1) > a')
all_tag.click()

문제 5.

Python 코드로 스타벅스 페이지에서 데이터를 가져올때, COFFEE_STORE 테이블에 바로 입력하도록 수정하세요.

- 데이터 세트: 매장 이름, 매장이 위치한 구 이름, 매장 주소, 위도, 경도
- 필요한 데이터를 한세트씩 가져와서 COFFEE_STORE 테이블에 각각INSERT 하도록 합니다.
- 입력된 데이터의 총 갯수를 쿼리하여 결과를 확인합니다.
- 입력된 데이터 상위 10개를 쿼리하여 결과를 확인합니다.

In [58]:
req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')
stores = soup.find('ul', 'quickSearchResultBoxSidoGugun').find_all('li')

In [59]:
len(stores)

610

In [60]:
insert_starbucks_sql = """
insert into coffee_store (brand, name, gu_name, address, lat, lng) values(1, %s, %s, %s, %s, %s)
"""

In [61]:
# 매장이름, 주소, 위도, 경도
for store in tqdm(stores):
    name = store.find('strong').text.strip()
    address = store.find('p').text.strip().replace('1522-3232', '')
    gu_name = address.split()[1]
    lat = store['data-lat']
    lng = store['data-long']
    mycursor.execute(insert_starbucks_sql, (name, gu_name, address, lat, lng))
    print(name, '\t', address, '\t', lat, '\t', lng)
    conn.commit()
    

  0%|          | 0/610 [00:00<?, ?it/s]

역삼아레나빌딩 	 서울특별시 강남구 언주로 425 (역삼동) 	 37.501087 	 127.043069
논현역사거리 	 서울특별시 강남구 강남대로 538 (논현동) 	 37.510178 	 127.022223
신사역성일빌딩 	 서울특별시 강남구 강남대로 584 (논현동) 	 37.5139309 	 127.0206057
국기원사거리 	 서울특별시 강남구 테헤란로 125 (역삼동) 	 37.499517 	 127.031495
대치재경빌딩R 	 서울특별시 강남구 남부순환로 2947 (대치동) 	 37.494668 	 127.062583
봉은사역 	 서울특별시 강남구 봉은사로 619 (삼성동) 	 37.515000 	 127.063196
압구정윤성빌딩 	 서울특별시 강남구 논현로 834 (신사동) 	 37.5227934 	 127.0286009
코엑스별마당 	 서울특별시 강남구 영동대로 513 (삼성동) 	 37.510150 	 127.060275
삼성역섬유센터R 	 서울특별시 강남구 테헤란로 518 (대치동) 	 37.507750 	 127.060651
압구정R 	 서울특별시 강남구 언주로 861 (신사동) 	 37.5273669 	 127.033061
수서역R 	 서울특별시 강남구 광평로 281 (수서동) 	 37.488008 	 127.102650
양재강남빌딩R 	 서울특별시 강남구 남부순환로 2621 (도곡동) 	 37.485192 	 127.036685
선릉동신빌딩R 	 서울특별시 강남구 테헤란로 409 (삼성동) 	 37.505321 	 127.050409
봉은사로선정릉 	 서울특별시 강남구 봉은사로 446 (삼성동) 	 37.511293 	 127.048409
강남오거리 	 서울특별시 강남구 봉은사로2길 39 (역삼동) 	 37.502117 	 127.026672
스타필드코엑스몰R 	 서울특별시 강남구 영동대로 513 (삼성동) 코엑스 A106호 	 37.50999 	 127.061455
강남구청정문 	 서울특별시 강남구 학동로 419 (청담동) 	 3

In [62]:
driver.quit()

제출 5.
- 스타벅스 데이터 관련 코드 & 실행 결과 (ipynb)

In [63]:
mycursor.execute('select count(name) from coffee_store')
print(mycursor.fetchall())

[(610,)]


In [64]:
mycursor.execute('SELECT * FROM coffee_store LIMIT 10')
rows = mycursor.fetchall()
columns = [column[0] for column in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
df

,id,brand,name,gu_name,address,lat,lng
0,1,1,역삼아레나빌딩,강남구,서울특별시 강남구 언주로 425 (역삼동),37.5011,127.043
1,2,1,논현역사거리,강남구,서울특별시 강남구 강남대로 538 (논현동),37.5102,127.022
2,3,1,신사역성일빌딩,강남구,서울특별시 강남구 강남대로 584 (논현동),37.5139,127.021
3,4,1,국기원사거리,강남구,서울특별시 강남구 테헤란로 125 (역삼동),37.4995,127.031
4,5,1,대치재경빌딩R,강남구,서울특별시 강남구 남부순환로 2947 (대치동),37.4947,127.063
5,6,1,봉은사역,강남구,서울특별시 강남구 봉은사로 619 (삼성동),37.5150,127.063
6,7,1,압구정윤성빌딩,강남구,서울특별시 강남구 논현로 834 (신사동),37.5228,127.029
7,8,1,코엑스별마당,강남구,서울특별시 강남구 영동대로 513 (삼성동),37.5102,127.060
8,9,1,삼성역섬유센터R,강남구,서울특별시 강남구 테헤란로 518 (대치동),37.5078,127.061
9,10,1,압구정R,강남구,서울특별시 강남구 언주로 861 (신사동),37.5274,127.033


문제 6.

Python 코드로 이디야 페이지에서 데이터를 가져올때, COFFEE_STORE 테이블에 바로 입력하도록 수정하세요.

- 데이터 세트 : 매장 이름, 매장이 위치한 구 이름, 매장 주소, 위도, 경도
- 이디야 페이지에서 검색에 사용할 구 이름은 COFFEE_STORE 에서 중복을 제거하는 쿼리를 사용하여 가져와서 {‘서울 ‘ + 구이름} 형식으로 변환하여 사용하도록 합니다.
- 필요한 데이터를 한 세트씩 가져와서 COFFEE_STORE 테이블에 각각 INSERT 하도록 합니다.
- (주의) COFFEE_STORE 테이블에 입력할 구 이름은 {‘서울 ‘} 이 제거된 구 이름입니다.
- 입력된 데이터의 총 갯수를 쿼리하여 결과를 확인합니다.
- 입력된 데이터 상위 10개를 쿼리하여 결과를 확인합니다.

In [65]:
mycursor.execute('select distinct gu_name from coffee_store')
rows = mycursor.fetchall()
gu_list = []
for row in rows:
    gu_list.append('서울 ' + row[0])

gu_list

['서울 강남구',
 '서울 강북구',
 '서울 강서구',
 '서울 관악구',
 '서울 광진구',
 '서울 금천구',
 '서울 노원구',
 '서울 도봉구',
 '서울 동작구',
 '서울 마포구',
 '서울 서대문구',
 '서울 서초구',
 '서울 성북구',
 '서울 송파구',
 '서울 양천구',
 '서울 영등포구',
 '서울 은평구',
 '서울 종로구',
 '서울 중구',
 '서울 강동구',
 '서울 구로구',
 '서울 동대문구',
 '서울 성동구',
 '서울 용산구',
 '서울 중랑구']

제출 6.
- 이디야 데이터 관련 코드 & 실행 결과 (ipynb)

In [66]:
ediya_url = 'https://ediya.com/contents/find_store.html'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(ediya_url)
driver.maximize_window()

In [67]:
# 주소 탭 클릭
driver.find_element(By.CSS_SELECTOR, '#contentWrap > div.contents > div > div.store_search_pop > ul > li:nth-child(2) > a').click()

In [68]:
insert_ediya_sql = """
insert into coffee_store (brand, name, gu_name, address, lat, lng) values(2, %s, %s, %s, %s, %s)
"""

In [69]:

for ele in tqdm(gu_list):
    # 주소 입력
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#keyword'))).click()
    driver.find_element(By.CSS_SELECTOR, '#keyword').clear()
    driver.find_element(By.CSS_SELECTOR, '#keyword').send_keys(ele)
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#keyword_div > form > button'))).click()
    
    # 뷰티풀 숲 활용        
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#placesList > li')))
    
    # 페이지가 갱신되었는지 확인
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#map > div:nth-child(1) > div > div:nth-child(6) > div:nth-child(1) > div > div > div.st_info_con > div.info_store_info > dl > dd:nth-child(1) > p')))
    test_ele = driver.find_element(By.CSS_SELECTOR, '#map > div:nth-child(1) > div > div:nth-child(6) > div:nth-child(1) > div > div > div.st_info_con > div.info_store_info > dl > dd:nth-child(1) > p')
    
    if test_ele.text == ele:           
        temp = driver.page_source
        soup = BeautifulSoup(temp, 'html.parser')
        stores = soup.find(id='placesList').find_all('li')
    else:
        time.sleep(2)
        temp = driver.page_source
        soup = BeautifulSoup(temp, 'html.parser')
        stores = soup.find(id='placesList').find_all('li')
    
    # 매장 정보 수집        
    for store in stores:            
        name = store.find('dl').find('dt').text
        address = store.find('dl').find('dd').text
        gu_name = address.split()[1]
        map_raw = store.find('a').get('onclick')                       
        
        pattern = re.compile(r"panLatTo\((.+?),(.+?),(.+?)\)")
        if pattern.match(map_raw):
            lng = pattern.match(map_raw).group(1)
            lat = pattern.match(map_raw).group(2)
            
            lng = lng.split("'")[1]
            lng = float(lng)
            
            lat = lat.split("'")[1]
            lat = float(lat)
        mycursor.execute(insert_ediya_sql, (name, gu_name, address, lat, lng))
        print(name, '\t', address, '\t', lat, '\t', lng)
        conn.commit()

  0%|          | 0/25 [00:00<?, ?it/s]

강남YMCA점 	 서울 강남구 논현동 	 0.0 	 0.0
강남구청역아이티웨딩점 	 서울 강남구 학동로 338 (논현동, 강남파라곤) 	 37.51654171724045 	 127.0401601992311
강남논현학동점 	 서울 강남구 논현로131길 28 (논현동) 	 37.51408005446769 	 127.02810578707653
강남대치점 	 서울 강남구 역삼로 415 (대치동, 성진빌딩) 	 37.50133876179308 	 127.05242928262568
강남도산점 	 서울 강남구 도산대로37길 20 (신사동) 	 37.50133876179308 	 127.05242928262568
강남율현점 	 서울 강남구 밤고개로21길 8 (율현동, 세곡프라자) 	 37.4735487656314 	 127.107863694133
강남자곡점 	 서울 강남구 자곡로 180 (자곡동, 강남유탑유블레스) 	 37.473606114795814 	 127.1031720971561
개포동역점 	 서울 강남구 개포로82길 11 (개포동, 삼우빌딩) 	 37.488604436046536 	 127.06742226515867
건설회관점 	 서울 강남구 언주로 711 (논현동) 	 37.488604436046536 	 127.06742226515867
대청역점 	 서울 강남구 개포로109길 34 (개포동, 대청프라자) 	 37.49418891219431 	 127.07764131137398
도곡2동점 	 서울 강남구 논현로 142 (도곡동, 동현빌딩) 	 37.482353824702095 	 127.04304591726503
도산사거리점 	 서울 강남구 논현동  	 37.482353824702095 	 127.04304591726503
삼성대치점 	 서울 강남구 삼성로64길 32 (대치동, 주성빌딩) 	 37.50043704079942 	 127.06310437319587
삼성동점 	 서울 강남구 테헤란로88길 10 (대치동) 	 37.50043704079942 	 127.0

In [72]:
driver.quit()

In [70]:
mycursor.execute('select count(name) from coffee_store where brand=2')
print(mycursor.fetchall())

[(616,)]


In [71]:
mycursor.execute('select * from coffee_store where brand=2 limit 10')
rows = mycursor.fetchall()
columns = [column[0] for column in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
df

,id,brand,name,gu_name,address,lat,lng
0,611,2,강남YMCA점,강남구,서울 강남구 논현동,0.0000,0.000
1,612,2,강남구청역아이티웨딩점,강남구,"서울 강남구 학동로 338 (논현동, 강남파라곤)",37.5165,127.040
2,613,2,강남논현학동점,강남구,서울 강남구 논현로131길 28 (논현동),37.5141,127.028
3,614,2,강남대치점,강남구,"서울 강남구 역삼로 415 (대치동, 성진빌딩)",37.5013,127.052
4,615,2,강남도산점,강남구,서울 강남구 도산대로37길 20 (신사동),37.5013,127.052
5,616,2,강남율현점,강남구,"서울 강남구 밤고개로21길 8 (율현동, 세곡프라자)",37.4735,127.108
6,617,2,강남자곡점,강남구,"서울 강남구 자곡로 180 (자곡동, 강남유탑유블레스)",37.4736,127.103
7,618,2,개포동역점,강남구,"서울 강남구 개포로82길 11 (개포동, 삼우빌딩)",37.4886,127.067
8,619,2,건설회관점,강남구,서울 강남구 언주로 711 (논현동),37.4886,127.067
9,620,2,대청역점,강남구,"서울 강남구 개포로109길 34 (개포동, 대청프라자)",37.4942,127.078


문제 7.

Python 코드에서 다음의 데이터를 쿼리를 사용하여 조회하세요.

- 스타벅스 매장 주요 분포 지역 (매장수가 많은 상위 5개 구이름, 매장 개수 출력)
- 이디야 매장 주요 분포 지역 (매장수가 많은 상위 5개 구이름, 매장 개수 출력)
- 구별 브랜드 각각의 매장 개수 조회 (구이름, 브랜드이름, 매장 개수 출력)
- 구별 브랜드 각각의 매장 개수 조회 (구이름, 스타벅스 매장 개수, 이디야 매장 개수 출력)

제출 7.
- 관련 코드 & 실행 결과 (ipynb)

In [74]:
# 스타벅스 매장 주요 분포 지역 (매장수가 많은 상위 5개 구이름, 매장 개수 출력)
sql = """
select gu_name, count(gu_name) 매장수
from coffee_store
where brand=1
group by gu_name
order by 매장수 desc
limit 5
"""
mycursor.execute(sql)

rows = mycursor.fetchall()
columns = [column[0] for column in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
df


,gu_name,매장수
0,강남구,90
1,중구,54
2,서초구,48
3,영등포구,42
4,종로구,40


In [75]:
# 이디야 매장 주요 분포 지역 (매장수가 많은 상위 5개 구이름, 매장 개수 출력)
sql = """
select gu_name, count(gu_name) 매장수
from coffee_store
where brand=2
group by gu_name
order by 매장수 desc
limit 5
"""
mycursor.execute(sql)

rows = mycursor.fetchall()
columns = [column[0] for column in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
df

,gu_name,매장수
0,영등포구,40
1,강남구,39
2,송파구,35
3,마포구,31
4,강서구,31


In [81]:
# 구별 브랜드 각각의 매장 개수 조회 (구이름, 브랜드이름, 매장 개수 출력)
sql = """
select s.gu_name 구, b.name 브랜드, count(*) 매장수
from coffee_store s
join coffee_brand b on s.brand = b.id
group by s.gu_name, b.name
order by gu_name
"""

mycursor.execute(sql)

rows = mycursor.fetchall()
columns = [column[0] for column in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
df

,구,브랜드,매장수
0,강남구,EDIYA,39
1,강남구,STARBUCKS,90
2,강동구,EDIYA,23
3,강동구,STARBUCKS,17
4,강북구,EDIYA,12
5,강북구,STARBUCKS,6
6,강서구,EDIYA,31
7,강서구,STARBUCKS,27
8,관악구,EDIYA,26
9,관악구,STARBUCKS,12


In [96]:
# 구별 브랜드 각각의 매장 개수 조회 (구이름, 스타벅스 매장 개수, 이디야 매장 개수 출력)
sql = """
select s.gu_name, s.count STARBUCKS, e.count EDIYA 
       from (select gu_name, count(*) count from coffee_store where brand = 1 group by gu_name) s, 
            (select gu_name, count(*) count from coffee_store where brand = 2 group by gu_name) e 
       where s.gu_name = e.gu_name
"""

mycursor.execute(sql)

rows = mycursor.fetchall()
columns = [column[0] for column in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
df

,gu_name,STARBUCKS,EDIYA
0,강남구,90,39
1,강북구,6,12
2,강서구,27,31
3,관악구,12,26
4,광진구,18,20
5,금천구,13,19
6,노원구,14,20
7,도봉구,6,22
8,동작구,11,21
9,마포구,36,31


제출 7.
- 관련 코드 & 실행 결과 (ipynb)

문제 8.

시각화 프로젝트를 위하여 다음의 규칙으로 쿼리하여 CSV 파일로 저장합니다. (Python 코드로 작업)

- 전체 데이터를 가져오는데, 각 스타벅스 매장별로 이디야 전체 매장정보가 매칭되어 있어야 합니다. (정렬 : s_id, e_id 순)
- 다음의 형식으로 저장되어야 합니다. (브랜드 이름, 칼럼 명 주의)
- 데이터 프레임 출력을 해주세요. 데이터 프레임 미출력시 감점입니다.

In [104]:
sql = """
select *
from (select s.id s_id, b.name s_brand, s.name s_name, s.gu_name s_gu, s.address s_address, s.lat s_lat, s.lng s_lng 
      from coffee_store s, coffee_brand b where s.brand = 1 and s.brand=b.id) st,
     (select s.id e_id, b.name e_brand, s.name e_name, s.gu_name e_gu, s.address e_address, s.lat e_lat, s.lng e_lng
      from coffee_store s, coffee_brand b where s.brand = 2 and s.brand=b.id) ed
order by st.s_id, ed.e_id
"""
mycursor.execute(sql)

rows = mycursor.fetchall()
columns = [column[0] for column in mycursor.description]
df = pd.DataFrame(rows, columns=columns)
df

,s_id,s_brand,s_name,s_gu,s_address,s_lat,s_lng,e_id,e_brand,e_name,e_gu,e_address,e_lat,e_lng
0,1,STARBUCKS,역삼아레나빌딩,강남구,서울특별시 강남구 언주로 425 (역삼동),37.5011,127.043,611,EDIYA,강남YMCA점,강남구,서울 강남구 논현동,0.0000,0.000
1,1,STARBUCKS,역삼아레나빌딩,강남구,서울특별시 강남구 언주로 425 (역삼동),37.5011,127.043,612,EDIYA,강남구청역아이티웨딩점,강남구,"서울 강남구 학동로 338 (논현동, 강남파라곤)",37.5165,127.040
2,1,STARBUCKS,역삼아레나빌딩,강남구,서울특별시 강남구 언주로 425 (역삼동),37.5011,127.043,613,EDIYA,강남논현학동점,강남구,서울 강남구 논현로131길 28 (논현동),37.5141,127.028
3,1,STARBUCKS,역삼아레나빌딩,강남구,서울특별시 강남구 언주로 425 (역삼동),37.5011,127.043,614,EDIYA,강남대치점,강남구,"서울 강남구 역삼로 415 (대치동, 성진빌딩)",37.5013,127.052
4,1,STARBUCKS,역삼아레나빌딩,강남구,서울특별시 강남구 언주로 425 (역삼동),37.5011,127.043,615,EDIYA,강남도산점,강남구,서울 강남구 도산대로37길 20 (신사동),37.5013,127.052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375755,610,STARBUCKS,중화역,중랑구,서울특별시 중랑구 봉화산로 35,37.6017,127.078,1222,EDIYA,중랑교차로점,중랑구,서울 중랑구 동일로 683 (면목동),0.0000,0.000
375756,610,STARBUCKS,중화역,중랑구,서울특별시 중랑구 봉화산로 35,37.6017,127.078,1223,EDIYA,중랑묵동점,중랑구,"서울 중랑구 동일로 932 (묵동, 묵동자이아파트)",37.6134,127.078
375757,610,STARBUCKS,중화역,중랑구,서울특별시 중랑구 봉화산로 35,37.6017,127.078,1224,EDIYA,중랑역점,중랑구,서울 중랑구 망우로 198 (상봉동),0.0000,0.000
375758,610,STARBUCKS,중화역,중랑구,서울특별시 중랑구 봉화산로 35,37.6017,127.078,1225,EDIYA,중화동점,중랑구,서울 중랑구 동일로129길 1 (중화동),37.5994,127.080


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375760 entries, 0 to 375759
Data columns (total 14 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   s_id       375760 non-null  int64  
 1   s_brand    375760 non-null  object 
 2   s_name     375760 non-null  object 
 3   s_gu       375760 non-null  object 
 4   s_address  375760 non-null  object 
 5   s_lat      375760 non-null  float64
 6   s_lng      375760 non-null  float64
 7   e_id       375760 non-null  int64  
 8   e_brand    375760 non-null  object 
 9   e_name     375760 non-null  object 
 10  e_gu       375760 non-null  object 
 11  e_address  375760 non-null  object 
 12  e_lat      375760 non-null  float64
 13  e_lng      375760 non-null  float64
dtypes: float64(4), int64(2), object(8)
memory usage: 40.1+ MB


In [114]:
mycursor.close()
conn.close()

In [115]:
df.to_csv('./starbucks_ediya.csv', index=False, encoding='utf-8')

In [116]:
test = pd.read_csv('./starbucks_ediya.csv')
test.head()

,s_id,s_brand,s_name,s_gu,s_address,s_lat,s_lng,e_id,e_brand,e_name,e_gu,e_address,e_lat,e_lng
0,1,STARBUCKS,역삼아레나빌딩,강남구,서울특별시 강남구 언주로 425 (역삼동),37.5011,127.043,611,EDIYA,강남YMCA점,강남구,서울 강남구 논현동,0.0000,0.000
1,1,STARBUCKS,역삼아레나빌딩,강남구,서울특별시 강남구 언주로 425 (역삼동),37.5011,127.043,612,EDIYA,강남구청역아이티웨딩점,강남구,"서울 강남구 학동로 338 (논현동, 강남파라곤)",37.5165,127.040
2,1,STARBUCKS,역삼아레나빌딩,강남구,서울특별시 강남구 언주로 425 (역삼동),37.5011,127.043,613,EDIYA,강남논현학동점,강남구,서울 강남구 논현로131길 28 (논현동),37.5141,127.028
3,1,STARBUCKS,역삼아레나빌딩,강남구,서울특별시 강남구 언주로 425 (역삼동),37.5011,127.043,614,EDIYA,강남대치점,강남구,"서울 강남구 역삼로 415 (대치동, 성진빌딩)",37.5013,127.052
4,1,STARBUCKS,역삼아레나빌딩,강남구,서울특별시 강남구 언주로 425 (역삼동),37.5011,127.043,615,EDIYA,강남도산점,강남구,서울 강남구 도산대로37길 20 (신사동),37.5013,127.052


제출 8.
- 시각화 프로젝트 관련 코드 (ipynb), 결과 파일 (csv)

---